# Data Science for Geoscience

Let's use some standard Machine Learning tools availble in Python packages to analyse some data.

# Machine Learning
Now we use our datasets to determine what tectonomagmatic parameters are related to ore formation. We use Support Vector Machines and Random Forrest classification methods. This notebook shows the two methods of data analysis, either looking at the general area of ore formation, or considering each ore formation point individually.


## Area of formation (Partioned Data)

#### Split the data into testing and training sets.
With the "training" set we learn which parameters are important, and we test the validity of this with the "testing" set.

In [13]:
import pandas
import numpy as np

#Import the tools for machine learning
from sklearn import cross_validation
from sklearn.svm import SVC
#from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

#Import data processing tools
#import pickle
#import matplotlib.mlab as ml

In [ ]:
bbb=pandas.read_csv("../examples/ml_data_panda.csv",index_col=0)
aaa=bbb.values

In [11]:
#Put the data into the correct format

#Get the parameters we want to include in the ML, indicies are from the List of Variables
params=[6,9,14,17] 
# params=[6,9,10,11,12,13,14,15,16,17]

#Recombine the features and the classification vectors. #Save the temporal-spatial parameters too for plotting purposes.
andesData = np.c_[preprocessing.scale(aaa[:,params]),aaa[:,5],aaa[:,18],aaa[:,20]]

#Do a 80/20 split of the data  to be used to make an example fit of the data
andesTrain, andesTest,  = cross_validation.train_test_split(\
       andesData, test_size=0.2, random_state=1)

print("Shape of data array: ", andesData.shape)

#Number of 'positive' and 'negative' examples
print("Positive (deposits) examples: ",np.shape(aaa[aaa[:,20]==1,:]))
print("Negative (non-deposits) examples: ",np.shape(aaa[aaa[:,20]==0,:]))

Shape of data array:  (221, 7)
Positive (deposits) examples:  (45, 21)
Negative (non-deposits) examples:  (176, 21)


In [14]:
#Make the classifier
clf = SVC(probability=True,verbose=True,cache_size=1000,class_weight=None)


#Now make a single classification for plotting and typicla results
#Train the classifier by fitting the parameters (features) to known results (targets/classes)
clf.fit(andesTrain[:,0:-3], andesTrain[:,-1])

print("Prediction-testing set (expected result):")
print(andesTest[:,-1])

print("Prediction of test (actual result):")
print(clf.predict(andesTest[:,0:-3]))

#Save the values of prediction
p=numpy.array(clf.predict_proba(andesTest[:,0:-3]))

#Get a single score out for the data
svmParams=clf.score(andesTest[:,0:-3], andesTest[:,-1])
print("Single result:")
print(svmParams)

# print clf.support_vectors_

###
#Now get a cross-fold validation score using all subsets of the data
scores = cross_validation.cross_val_score(clf, andesData[:,0:-3], andesData[:,-1], cv=5)

print("Cross fold validation results:")
print(scores)

#print clf.n_support_

[LibSVM]Prediction-testing set (expected result):
[ 1.  1.  1.  0.  1.  0.  0.  0.  1.  0.  1.  1.  0.  0.  0.  0.  1.  0.
  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.]
Prediction of test (actual result):
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Single result:
0.711111111111
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Cross fold validation results:
[ 0.82222222  0.84090909  0.86363636  0.77272727  0.79545455]
